In [ ]:
!pip install spacy-transformers xlsxwriter
!pip install pypdf

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import xlsxwriter
import spacy
from pypdf import PdfReader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!python -m spacy download en_core_web_trf

In [ ]:
spacy.prefer_gpu()
trf = spacy.load('en_core_web_trf')

In [ ]:
def get_ner(trf):
    ner_dict = {
       "NORP": "NATIONAL/RELIGIOUS/POLITICAL GROUP",
        "FAC": "ARCHITECTURE",
        "ORG": "ORGANIZATION",
        "GPE": "GEOPOLITICAL ENTITY",
        "LOC": "LOCATION",
    }
    reader = PdfReader("/kaggle/input/ner-app/file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    doc = trf(text)
    entity_freq = {}
    for ent in doc.ents:
        if ent.label_ not in ["CARDINAL", "DATE", "QUANTITY", "ORDINAL", "TIME", "PERCENT", "MONEY"]:
            if ent.text not in entity_freq:
                if ent.label_ in ner_dict.keys():
                  entity_freq[ent.text.replace("\n", "")] = (ner_dict[ent.label_], 1)
                else:
                  entity_freq[ent.text.replace("\n", "")] = (ent.label_, 1)
            else:
                if ent.label_ in ner_dict.keys():
                  entity_freq[ent.text.replace("\n", "")] = (ner_dict[ent.label_],entity_freq[ent.text.replace("\n", "")][1]+1)
                else:
                  entity_freq[ent.text.replace("\n", "")] = (ent.label_,entity_freq[ent.text.replace("\n", "")][1]+1)
    return sorted(entity_freq.items(), key=lambda x: x[1][1], reverse=True)

In [ ]:
def write_excel(data):

  # Create a new Excel file
  workbook = xlsxwriter.Workbook('/kaggle/working/data.xlsx')
  # Add a new worksheet to the Excel file
  worksheet = workbook.add_worksheet()

  # Write the data to the worksheet
  for row_num, row_data in enumerate(data):
      for col_num, col_data in enumerate(row_data):
          # Check if the third column is a link
          if col_num == 2 and 'http' in col_data:
              worksheet.write_url(row_num, col_num, col_data)
          else:
              worksheet.write(row_num, col_num, col_data)

  # Close the Excel file
  workbook.close()

In [ ]:
bing_dict = {
        "PERSON": "Describe this person in 5 sentences: ",
        "NATIONAL/RELIGIOUS/POLITICAL GROUP": "Describe this national/religious/political group in 5 sentences: ",
        "ARCHITECTURE": "Describe this building in 5 sentences: ",
        "ORGANIZATION": "Describe this organization in 5 sentences: ",
        "GEOPOLITICAL ENTITY": "Describe this geopolitical entity in 5 sentences: ",
        "LOCATION": "Describe this location in 5 sentences:",
        "PRODUCT": "Describe this product in 5 sentences: ",
        "EVENT": "Explain this event in 5 sentences: ",
        "WORK_OF_ART": "Describe this work of art in 5 sentences:",
        "LAW": "Explain this law in 5 sentences: ",
        "LANGUAGE": "Explain this language in 5 sentences: "
    }

In [ ]:
def export_to_csv(data):
  new_data = []
  for item in data:
    name, tag = item[0], item[1][0]
    temp = (name, tag, f"https://www.bing.com/search?q={bing_dict[tag].replace(' ', '+')}+{name.replace(' ', '+')}")
    new_data.append(temp)
  write_excel(new_data)

In [ ]:
data = get_ner(trf)
export_to_csv(data)